In [1]:
concepts = [
    "bay",
    "beach",
    "birds",
    "boeing",
    "buildings",
    "city",
    "clouds",
    "face",
    "f-16",
    "helicopter",
    "mountain",
    "sky",
    "ships",
    "sunset",
    "sunrise",
    "ocean"
]

In [2]:
cat = "bay"

In [3]:
fname = "urls/{}.txt".format(cat)

In [5]:
with open(fname, encoding='utf8') as f:
    lines = f.readlines()
urls = [line.rsplit('\n')[0] for line in lines]

In [14]:
dirname = "data/{}".format(cat)

!mkdir -p $dirname

In [57]:
from concurrent import futures
import urllib.request

    

In [54]:
def get_one(dirname, urlidx, urls):
    urllib.request.urlretrieve(urls[urlidx], "{}/{:04d}.{}".format(dirname, urlidx, urls[urlidx].rsplit(".")[-1]))

In [45]:
# keep order and remove dup
def uniq_list(urllist):
    done = set()
    ret = []
    for url in urllist:
        if not url in done:
            ret.append(url)
            done.add(url)
    return ret

In [62]:
MAX_WORKERS = 10

def download_many(urlist, dirname):
    uniqurls = uniq_list(urlist)
    with futures.ThreadPoolExecutor(MAX_WORKERS) as executor:
        res = executor.map(lambda idx: get_one(dirname, idx, uniqurls), range(len(uniqurls)))
    _ = len(list(res))
    print("done")

In [ ]:
download_many(urls, dirname)

In [ ]:
3

In [38]:
a = set()

In [39]:
a.add(0)

In [40]:
a.add(1)

In [42]:
1 in a

True

In [30]:
get_one(dirname, 1, urls)

In [26]:
urlidx = 0

In [28]:
urllib.request.urlretrieve(urls[urlidx], "{}/{}.{}".format(dirname, urlidx+1, urls[urlidx].rsplit(".")[-1]))

('data/bay/1.jpg', <http.client.HTTPMessage at 0x7fc74c0c50f0>)

In [19]:
urls[0].rsplit(".")[-1]

'jpg'